In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

from CustomDataset.Data import (
    DSD100,
    data_split,
    dataloader
)
from model.Network import UNet

In [2]:
torch.manual_seed(100)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset = DSD100('../DSD100spectrogram')

In [5]:
for mixture, bass, drum, vocal, instrumental in dataset:
    print(mixture.shape)
    print(bass.shape)
    print(drum.shape)
    print(vocal.shape)
    print(instrumental.shape)
    break

torch.Size([1, 511, 127])
torch.Size([1, 511, 127])
torch.Size([1, 511, 127])
torch.Size([1, 511, 127])
torch.Size([1, 511, 127])


In [6]:
len(dataset)

10284

In [7]:
train_dataset, val_dataset = data_split(dataset, 0.7)

In [8]:
len(train_dataset)

7199

In [9]:
len(val_dataset)

3085

In [10]:
train_dataloader = dataloader(train_dataset, 128, shuffle = True)
val_dataloader = dataloader(val_dataset, 128, shuffle = False)

In [11]:
for mixture, bass, drum, vocal, instrumental in train_dataloader:
    print(mixture.shape)
    print(bass.shape)
    print(drum.shape)
    print(vocal.shape)
    print(instrumental.shape)
    break

torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])


UNet model

In [12]:
model = UNet().to(device)

In [13]:
summary(model, input_size = (1, 511, 127), batch_size = 128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [128, 16, 255, 63]             416
       BatchNorm2d-2         [128, 16, 255, 63]              32
         LeakyReLU-3         [128, 16, 255, 63]               0
           encoder-4         [128, 16, 255, 63]               0
            Conv2d-5         [128, 32, 127, 31]          12,832
       BatchNorm2d-6         [128, 32, 127, 31]              64
         LeakyReLU-7         [128, 32, 127, 31]               0
           encoder-8         [128, 32, 127, 31]               0
            Conv2d-9          [128, 64, 63, 15]          51,264
      BatchNorm2d-10          [128, 64, 63, 15]             128
        LeakyReLU-11          [128, 64, 63, 15]               0
          encoder-12          [128, 64, 63, 15]               0
           Conv2d-13          [128, 128, 31, 7]         204,928
      BatchNorm2d-14          [128, 128